In [ ]:
##Let's Convolute!
##Ok, so a word about the Architecture: 
##Ok, so we have 6 5x5 filters which we pass over the 28x28 grid, and after the ReLu and Pool, we get 6 12x12 matrices which will form our second layer. Then, we have 12 6x5x5 matrices which we slide over the 6 12x12 matrices in our second layer, and we compute 6 matrix dot products simultaneously and add the bias to get a single element in the output. After the ReLu and Pool, we will get out 12 4x4 matrices, which are then flattened and fed to our fully connected Neural Network. 


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist  
import math
from numpy import random

def sigmoid(x):
    return 1/(1+np.exp(-x))
def derivative_sigmoid(x):
    return np.exp(x)/(1+np.exp(x))**2
def softmax(x):
    total = 0
    output = [0.0]*len(x)
    for index in range(len(x)):
        output[index] = np.exp(x[index])
        total = total + output[index]
    output = output/total
    return output
def derivative_softmax(x):
    return np.multiply(softmax(x), [1]*len(x) - softmax(x))

def cross_correlation(matrix, kernel):
    output = np.array([[0.0]*(len(matrix[0])-len(kernel)+1)]*(len(matrix)-len(kernel)+1))
    for i in range(len(output)):
        for j in range(len(output[i])):
             output[i][j] = np.sum(np.multiply(np.array(matrix[i:i+len(kernel), j:j+len(kernel)]), kernel))
    return output 

def multiple_cross_correlation(matrix, kernel):
    output = [[0.0]*(len(matrix[0][0])-len(kernel[0])+1)]*(len(matrix[0])-len(kernel[0])+1)
    for i in range(len(matrix)):
        output = output + cross_correlation(matrix[i], kernel[i])
    return output


def pool(matrix):
    output_length = int(len(matrix)/2)
    output = np.array([[0.0]*output_length]*output_length)
    for i in range(len(output)):
        for j in range(len(output[i])):
            output[i][j] = np.max(np.array(matrix[2*i:2*i+2, 2*j:2*j+2]))
    return output

def ReLu(x):
    for i in range(len(x)):
        for j in range(len(x[i])):
            if(x[i][j] < 0 or x[i][j] == 0):
                x[i][j] = 0
    return x

def derivative_ReLu(x):
    for i in range(len(x)):
        for j in range(len(x[i])):
            if(x[i][j] < 0 or x[i][j] == 0):
                x[i][j] = 0
            else:
                x[i][j] = 1
    return x

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

weights_0_to_1 = random.normal(0, math.sqrt(2/784), 150).reshape(6, 5, 5)
bias_0_to_1 =  random.normal(0, math.sqrt(2/784), 6)
weights_1_to_2 = random.normal(0, math.sqrt(2/864), 1800).reshape(12, 6, 5, 5)
bias_1_to_2 = random.normal(0, math.sqrt(2/864), 12)
weights_2_to_3 = random.normal(0, 1/math.sqrt(192), 136*192).reshape(136, 192)
bias_2_to_3 = random.normal(0, 1/math.sqrt(192), 136)
weights_3_to_4 = random.normal(0, 1/math.sqrt(136), 1360).reshape(10, 136)
bias_3_to_4 = random.normal(0, 1/math.sqrt(136), 10)  

print(weights_0_to_1)
print(weights_1_to_2)
print(weights_2_to_3)
print(weights_3_to_4)

weights_3_to_4_m_matrix = np.array([[0.0]*136]*10)
weights_3_to_4_v_matrix = np.array([[0.0]*136]*10)
bias_3_to_4_m_matrix = np.array([0.0]*10)
bias_3_to_4_v_matrix = np.array([0.0]*10)
weights_2_to_3_m_matrix = np.array([[0.0]*192]*136)
weights_2_to_3_v_matrix = np.array([[0.0]*192]*136)
bias_2_to_3_m_matrix = np.array([0.0]*136)
bias_2_to_3_v_matrix = np.array([0.0]*136)
weights_1_to_2_m_matrix = np.array([[[[0.0]*5]*5]*6]*12)
weights_1_to_2_v_matrix = np.array([[[[0.0]*5]*5]*6]*12)
bias_1_to_2_m_matrix = np.array([0.0]*12)
bias_1_to_2_v_matrix = np.array([0.0]*12)
weights_0_to_1_m_matrix = np.array([[[0.0]*5]*5]*6)
weights_0_to_1_v_matrix = np.array([[[0.0]*5]*5]*6)
bias_0_to_1_m_matrix = np.array([0.0]*6)
bias_0_to_1_v_matrix = np.array([0.0]*6)

elements_to_test_with = random.randint(59999, size=4800)

for r in range(1, 4801):
    S1 = []
    P1 = []
    C1 = []
    P2 = []
    C2 = []
    S2 = []
    standardized_test_element = (X_train[elements_to_test_with[r-1]] - np.mean(X_train[elements_to_test_with[r-1]]))/np.std(X_train[elements_to_test_with[r-1]]) 
    for i in range(len(weights_0_to_1)):
        S1.append(cross_correlation(standardized_test_element, weights_0_to_1[i]) + bias_0_to_1[i])
        C1.append(ReLu(cross_correlation(standardized_test_element, weights_0_to_1[i]) + bias_0_to_1[i]))
        P1.append(pool(ReLu(cross_correlation(standardized_test_element, weights_0_to_1[i]) + bias_0_to_1[i])))
    S1 = np.array(S1)
    C1 = np.array(C1)
    P1 = np.array(P1)

    for j in range(len(weights_1_to_2)):
        S2.append(multiple_cross_correlation(P1 , weights_1_to_2[j]) + bias_1_to_2[j])
        C2.append(ReLu(multiple_cross_correlation(P1 , weights_1_to_2[j]) + bias_1_to_2[j]))
        P2.append(pool(ReLu(multiple_cross_correlation(P1 , weights_1_to_2[j]) + bias_1_to_2[j]))) 
    S2 = np.array(S2)
    C2 = np.array(C2)
    P2 = np.array(P2) 

    layer_3 = sigmoid(np.matmul(weights_2_to_3 , P2.reshape(192,)) + bias_2_to_3)
    layer_4 = softmax(np.matmul(weights_3_to_4, layer_3) + bias_3_to_4)

    delta_4 = [0.0]*10
    accurate_output = [0]*10
    accurate_output[Y_train[elements_to_test_with[r-1]]] = 1 
    for layer_4_activation_index in range(len(layer_4)):
        delta_4[layer_4_activation_index] = -(accurate_output[layer_4_activation_index]/layer_4[layer_4_activation_index] - (1-accurate_output[layer_4_activation_index])/(1-layer_4[layer_4_activation_index]))*(derivative_softmax(np.matmul(weights_3_to_4, layer_3) + bias_3_to_4)[layer_4_activation_index])
    delta_3 = np.multiply(np.matmul(weights_3_to_4.T, delta_4), derivative_sigmoid(np.matmul(weights_2_to_3 , P2.reshape(192,)) + bias_2_to_3))

    for i in range(len(weights_3_to_4)):
        for j in range(len(weights_3_to_4[i])):
            a = layer_3[j]*delta_4[i]
            weights_3_to_4_m_matrix[i][j] = 0.9*weights_3_to_4_m_matrix[i][j] + (1-0.9)*a
            weights_3_to_4_v_matrix[i][j] = 0.999*weights_3_to_4_v_matrix[i][j] + (1-0.999)*(a**2)
            m_hat = weights_3_to_4_m_matrix[i][j]/(1-0.9**r)
            v_hat = weights_3_to_4_v_matrix[i][j]/(1-0.999**r)
            weights_3_to_4[i][j] = weights_3_to_4[i][j] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 
    for k in range(len(bias_3_to_4)):
        a = delta_4[k]
        bias_3_to_4_m_matrix[k] = 0.9*bias_3_to_4_m_matrix[k] + (1-0.9)*a
        bias_3_to_4_v_matrix[k] = 0.999*bias_3_to_4_v_matrix[k] + (1-0.999)*(a**2)
        m_hat = bias_3_to_4_m_matrix[k]/(1-0.9**r)
        v_hat = bias_3_to_4_v_matrix[k]/(1-0.999**r)
        bias_3_to_4[k] = bias_3_to_4[k] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 
    for l in range(len(weights_2_to_3)):
        for m in range(len(weights_2_to_3[l])):
            a = P2.reshape(192,)[m]*delta_3[l]
            weights_2_to_3_m_matrix[l][m] = 0.9*weights_2_to_3_m_matrix[l][m] + (1-0.9)*a
            weights_2_to_3_v_matrix[l][m] = 0.999*weights_2_to_3_v_matrix[l][m] + (1-0.999)*(a**2)
            m_hat = weights_2_to_3_m_matrix[l][m]/(1-0.9**r)
            v_hat = weights_2_to_3_v_matrix[l][m]/(1-0.999**r)
            weights_2_to_3[l][m] = weights_2_to_3[l][m] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 
    for n in range(len(bias_2_to_3)):
        a = delta_3[n] 
        bias_2_to_3_m_matrix[n] = 0.9*bias_2_to_3_m_matrix[n] + (1-0.9)*a
        bias_2_to_3_v_matrix[n] = 0.999*bias_2_to_3_v_matrix[n] + (1-0.999)*(a**2)
        m_hat = bias_2_to_3_m_matrix[n]/(1-0.9**r)
        v_hat = bias_2_to_3_v_matrix[n]/(1-0.999**r)
        bias_2_to_3[n] = bias_2_to_3[n] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 

    delta_2 = np.matmul(weights_2_to_3.T, delta_3)

    dLP2 = delta_2.reshape(12, 4, 4)
    dLC2 = np.array([[[0.0]*8]*8]*12)

    for i in range(len(P2)):
        for j in range(len(P2[i])):
            for k in range(len(P2[i][j])):
                for l in range(2*j, 2*j+2):
                    for m in range(2*k, 2*k+2):
                        if(P2[i][j][k] == C2[i][l][m] and P2[i][j][k] != 0):
                            dLC2[i][l][m] = dLP2[i][j][k] 

    dC2S2 =  np.array([[[0.0]*8]*8]*12)
    for i in range(len(dC2S2)):
        for j in range(len(dC2S2[i])):
            for k in range(len(dC2S2[i][j])):
                if(S2[i][j][k] > 0):
                    dC2S2[i][j][k] = 1

    dLS2 = np.array(np.multiply(dLC2, dC2S2))

    for m in range(len(bias_1_to_2)):
        a = np.sum(np.multiply(dLC2[m], dC2S2[m]))
        bias_1_to_2_m_matrix[m] = 0.9*bias_1_to_2_m_matrix[m] + (1-0.9)*a
        bias_1_to_2_v_matrix[m] = 0.999*bias_1_to_2_v_matrix[m] + (1-0.999)*(a**2)
        m_hat = bias_1_to_2_m_matrix[m]/(1-0.9**r)
        v_hat = bias_1_to_2_v_matrix[m]/(1-0.999**r)
        bias_1_to_2[m] = bias_1_to_2[m] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 

    for m in range(len(weights_1_to_2)):
        for n in range(len(weights_1_to_2[m])):
            for p in range(len(weights_1_to_2[m][n])):
                for q in range(len(weights_1_to_2[m][n][p])):
                    total = 0
                    for u in range(len(S2[m])):
                        for v in range(len(S2[m][u])):
                            total = total + dLS2[m][u][v]*P1[n][p+u][q+v]
                    a = total
                    weights_1_to_2_m_matrix[m][n][p][q] = 0.9*weights_1_to_2_m_matrix[m][n][p][q] + (1-0.9)*a
                    weights_1_to_2_v_matrix[m][n][p][q] = 0.999*weights_1_to_2_v_matrix[m][n][p][q] + (1-0.999)*(a**2)
                    m_hat = weights_1_to_2_m_matrix[m][n][p][q]/(1-0.9**r)
                    v_hat = weights_1_to_2_v_matrix[m][n][p][q]/(1-0.999**r)
                    weights_1_to_2[m][n][p][q] = weights_1_to_2[m][n][p][q] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 

    dS2P1 = np.array([[[[[[0.0]*8]*8]*12]*12]*12]*6)

    for m in range(12):
        for u in range(8):
            for v in range(8):
                for n in range(len(dS2P1)):
                    for t in range(u, u+5):
                        for s in range(v, v+5): 
                            dS2P1[n][t][s][m][u][v] = weights_1_to_2[m][n][t-u][s-v]

    dLP1 = np.array([[[0.0]*12]*12]*6) 

    for n in range(len(dLP1)):
        for t in range(len(dLP1[n])):
            for s in range(len(dLP1[n][t])): 
                dLP1[n][t][s] = np.sum(np.multiply(dLS2, dS2P1[n][t][s]))

    dLC1 = np.array([[[0.0]*24]*24]*6)

    for i in range(len(dLP1)):
        for j in range(len(dLP1[i])):
            for k in range(len(dLP1[i][j])):
                for l in range(2*j, 2*j+2):
                    for m in range(2*k, 2*k+2):
                        if(P1[i][j][k] == C1[i][l][m] and P1[i][j][k] != 0):
                            dLC1[i][l][m] = dLP1[i][j][k]

    dC1S1 = np.array([[[0.0]*24]*24]*6)

    for i in range(len(dC1S1)):
        for j in range(len(dC1S1[i])):
            for k in range(len(dC1S1[i][j])):
                if(S1[i][j][k] > 0):
                    dC1S1[i][j][k] = 1

    dLS1 = np.array(np.multiply(dLC1, dC1S1))

    for i in range(len(bias_0_to_1)):
        a = np.sum(np.multiply(dLC1[i], dC1S1[i])) 
        bias_0_to_1_m_matrix[i] = 0.9*bias_0_to_1_m_matrix[i] + (1-0.9)*a
        bias_0_to_1_v_matrix[i] = 0.999*bias_0_to_1_v_matrix[i] + (1-0.999)*(a**2)
        m_hat = bias_0_to_1_m_matrix[i]/(1-0.9**r)
        v_hat = bias_0_to_1_v_matrix[i]/(1-0.999**r)
        bias_0_to_1[i] = bias_0_to_1[i] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 

    for n in range(len(weights_0_to_1)):
        for s in range(len(weights_0_to_1[n])):
            for h in range(len(weights_0_to_1[n][s])):
                total = 0 
                for i in range(24):
                    for j in range(24):
                        total = total + dLS1[n][i][j]*standardized_test_element[s+i][h+j]
                a = total
                weights_0_to_1_m_matrix[n][s][h] = 0.9*weights_0_to_1_m_matrix[n][s][h] + (1-0.9)*a
                weights_0_to_1_v_matrix[n][s][h] = 0.999*weights_0_to_1_v_matrix[n][s][h] + (1-0.999)*(a**2)
                m_hat = weights_0_to_1_m_matrix[n][s][h]/(1-0.9**r)
                v_hat = weights_0_to_1_v_matrix[n][s][h]/(1-0.999**r)
                weights_0_to_1[n][s][h] = weights_0_to_1[n][s][h] - (0.001 * 1/(math.sqrt(v_hat) + 10e-8) * m_hat) 

print(weights_0_to_1)
print(bias_0_to_1)
print(weights_1_to_2)
print(bias_1_to_2)
print(weights_2_to_3)
print(bias_2_to_3)
print(weights_3_to_4)
print(bias_3_to_4)

accuracy = 0
for r in range(len(X_test)):
    standardized_test_element = (X_test[r] - np.mean(X_test[r]))/np.std(X_test[r])
    P1 = [] 
    P2 = []
    for i in range(len(weights_0_to_1)): 
        P1.append(pool(ReLu(cross_correlation(standardized_test_element, weights_0_to_1[i]) + bias_0_to_1[i])))
    P1 = np.array(P1)
    for j in range(len(weights_1_to_2)):
        P2.append(pool(ReLu(multiple_cross_correlation(P1 , weights_1_to_2[j]) + bias_1_to_2[j]))) 
    P2 = np.array(P2)
    layer_3 = sigmoid(np.matmul(weights_2_to_3 , P2.reshape(192,)) + bias_2_to_3)
    layer_4 = softmax(np.matmul(weights_3_to_4, layer_3) + bias_3_to_4)
    if(layer_4[Y_test[r]] == np.amax(layer_4)): 
        accuracy = accuracy + 1     
print(accuracy)     

[[[ 2.73510294e-02  2.92748471e-02 -4.03179470e-02 -4.08664814e-02
   -6.84591712e-02]
  [-2.88081513e-02  1.61460627e-02  7.87424603e-03  1.08541957e-01
    7.21390818e-03]
  [-2.53120265e-02 -7.15540331e-02 -1.07609303e-02 -5.78024011e-03
   -2.82166928e-02]
  [-1.50378480e-02 -3.94473605e-02 -6.17013669e-02  3.05200529e-02
    2.52339316e-02]
  [ 3.69566086e-02 -7.43829254e-02  4.89688252e-02 -5.37588406e-02
    1.17946931e-02]]

 [[-3.19428972e-02  5.10649910e-02  7.70679003e-02  6.41056098e-03
    1.17754416e-01]
  [ 1.78141305e-02  3.25250612e-02  3.30814158e-02 -1.06958684e-02
   -6.79816418e-02]
  [-6.61133882e-04 -2.10227353e-02  1.17576411e-01 -7.51030824e-02
   -4.71757914e-02]
  [-3.93060684e-02 -3.21200345e-02 -7.07762727e-02  1.31759706e-02
    7.57046041e-02]
  [-1.23372166e-02  8.76623828e-02 -2.51254678e-02  3.29995058e-02
   -1.99202603e-02]]

 [[ 1.80001677e-02 -2.64470569e-02  4.35540519e-02 -4.83548033e-02
   -6.82210487e-02]
  [-3.93742741e-02  8.43481574e-02 -8.5